In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

examples = [
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
]
example_template = """
영화 제목: {input}
이모티콘: {output}
"""
few_shot_prompt = FewShotPromptTemplate.from_examples(
    examples=examples,
    example_prompt=PromptTemplate.from_template(example_template),
    prefix="다음은 영화 제목을 세 개의 이모티콘으로 표현한 예시입니다.",
    suffix="영화 제목: {question}\n이모티콘:",
    input_variables=["question"],
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=few_shot_prompt,
    verbose=True,
)

response_1 = chain.predict(question="탑건")
print("탑건:", response_1)

response_2 = chain.predict(question="대부")
print("대부:", response_2)

response_3 = chain.predict(question="내가 처음에 물어본 영화가 뭐였지?")
print("처음 질문한 영화:", response_3)
